<a href="https://colab.research.google.com/github/makeflower99/test/blob/master/%EA%B0%80%EC%84%A41_%EC%8A%A4%ED%8C%8C%EB%A5%B4%ED%83%80%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium
!pip install konlpy

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [662 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,344 kB]
Get:13 http://security.ubuntu.

In [ ]:
from collections import Counter
from konlpy.tag import Okt
import nltk
import re
import pandas as pd
import time
from nltk import pos_tag
#크롤링한 결과 불러오기
# 'Sparta.csv'파일과 '불용어리스트.csv'는 파일위치에서 최상위부분에 위치하여야한다.
#파일 넣는 방법 좌측에 폴더를 클릭한 후
# ..(상위폴더로 이동)을 선택한 후 파일 업로드 버튼을 눌러 업로드 한다.
# 저장된 파일은 content폴더에 위치해있습니다.

#스파르타 크롤링 결과
df1= pd.read_csv('/Sparta.csv')
df1.columns=['title']
list1 = df1['title'].tolist()
# 불용어 리스트 정의
df2=pd.read_csv('/불용어리스트.csv')
df2.columns=['name']
list2 = df2['name'].tolist()


# 추가적인 불용어 정의
additional_stopwords = ['[', ']', '(', ')', ',', '.', '!', '?', '위한', '위']

# 불용어 리스트 결합
stopwords = list2 + additional_stopwords

# 띄어쓰기를 기준으로 단어 분리 및 불용어 제거
tokens = []
for title in list1:
    title = re.sub(r"[^가-힣\s]", "", title)  # 한글과 공백을 제외한 모든 문자 제거
    words = title.split()  # 띄어쓰기를 기준으로 단어 분리
    for word in words:
        if word not in stopwords:
            tokens.append(word)

# 가장 많이 나타나는 단어 추출
word_count = Counter(tokens)
most_common_words = word_count.most_common()

# 형태소 분석기 초기화
okt = Okt()

# 제외하고 싶은 단어 리스트 정의
exclude_words = ['다시보기','월간','만들기']

# 명사에 포함될 단어 리스트 정의
include_words = ['챌린지','시각화']

# 추출된 단어들 중 명사 처리
nouns_only_korean = {}
for word, count in most_common_words:

    # 제외 대상 단어인 경우 건너뛰기
    if word in exclude_words:
        continue
    # 명사에 포함할 단어 예외 처리
    if word in include_words:
        nouns_only_korean[word] = count
        continue

    pos_list = okt.pos(word)

    # 모든 형태소가 명사인 경우
    if all(pos == 'Noun' for _, pos in pos_list):
        nouns_only_korean[word] = count

    # 하나라도 명사가 아닌 형태소가 있는 경우
    else:
        for morph, pos in pos_list:
            if pos == 'Noun':
                if morph in nouns_only_korean:
                    # 이미 있는 명사는 빈도수를 더함
                    nouns_only_korean[morph] += count
                else:
                    # 새 명사는 추가
                    nouns_only_korean[morph] = count

# nltk에서 필요한 자원 다운로드
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# 데이터 불러오기
df1 = pd.read_csv('/Sparta.csv')
df1.columns = ['title']
list1 = df1['title'].tolist()

# 추가적인 불용어 정의 (영어 버전)
stopwords_english = ['[', ']', '(', ')', ',', '.', '!', '?', 'and', 'the']

# 텍스트를 소문자로 변환하지 않고 불용어 제거
filtered_text = ''
for title in list1:
    title = re.sub(r"[^a-zA-Z\s]", "", title)  # 영문자와 공백을 제외한 모든 문자 제거
    words_english = title.split()  # 단어 분리 (공백을 기준으로)
    filtered_words_english = []
    for word in words_english:
        if word not in stopwords_english:
            filtered_words_english.append(word)
    filtered_text += ' '.join(filtered_words_english) + ' '

# 가장 많이 나타나는 단어 추출
word_count_english = Counter(filtered_text.split())
most_common_words_english = word_count_english.most_common()

# 영어 형태소 분석 및 명사 추출
nouns_only_english = {}
for word, count in most_common_words_english:
    tagged_word = pos_tag([word])
    if tagged_word[0][1].startswith('NN'):  # 명사(NN, NNP, NNS 등)인 경우
        nouns_only_english[word] = count


nouns_only_combined = nouns_only_english.copy()  # nouns_only를 복사하여 새로운 딕셔너리 생성
nouns_only_combined.update(nouns_only_korean)

sorted_nouns_only_combined = sorted(nouns_only_combined.items(),key=lambda x: x[1], reverse=True)

result_df = pd.DataFrame(sorted_nouns_only_combined, columns=['word', 'count'])
result_df.to_csv('Sparta_result.csv',index=False)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
